In [15]:
## Library
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
def searchHTML(URL):
    #Request HTML & beautify it
    page = requests.get(URL)
    soup = bs(page.content)
    
    #Find right data
    results_numbers = soup.find_all("span", {'class' : 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC'})
    results_hash = soup.find_all("a", {'class' : 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK'})
    
    return results_numbers, results_hash

def makeFrame(hash, time, BTC, USD):
    dataF = {'Hash' : hash, 'Time' : time, 'BTC-value' : BTC, 'USD-value' : USD}
    frameUpdate = pd.DataFrame(dataF)
        
    return frameUpdate

def makeArrays(numbers, hash):
    only_time = [content.get_text() for content in numbers[0::3]]
    only_BTC = [content.get_text() for content in numbers[1::3]]
    only_USD = [content.get_text() for content in numbers[2::3]]
    only_hash = [content.get_text() for content in hash]
    
    return only_hash, only_time, only_BTC, only_USD

def fetchCurrent(URL):
    #HTML-doorzoeken en alles ophalen
    numbers, hash = searchHTML(URL)

    #Make arrays of only required values
    only_hash, only_time, only_BTC, only_USD = makeArrays(numbers, hash)

    #Make dataframe
    frame = makeFrame(only_hash, only_time, only_BTC, only_USD)
    
    #unique = frame['Time'].nunique()
    
    return frame
    



In [3]:
def liveCall():
    #Eenmalig eventjes een lege dataframe maken om te kunnen appenden
    dataF = {'Hash' : [], 'Time' : [], 'BTC-value' : [], 'USD-value' : []}
    frame = pd.DataFrame(data=dataF)
    
    while 1:
        URL = "https://www.blockchain.com/btc/unconfirmed-transactions"
        frame_new = fetchCurrent(URL)
        
        liveCall.appendedDF = frame.append(frame_new).drop_duplicates()
        frame = liveCall.appendedDF
        
        #print(unique)


liveCall()        


KeyboardInterrupt: 

In [4]:
print(liveCall.appendedDF)

                                                 Hash   Time  \
0   a85a0057addc60deddd5052b95c6f6a6d601b8c1025404...  20:58   
1   6bc4a94902dcb764d546eb7f96842211b7c0e2d8067f11...  20:58   
2   2088d2ea6445ecab7e12809554ee493586b5304c310530...  20:58   
3   25f7b4cc93691b1233210eabad689a84cb303878261c0a...  20:58   
4   9adb361849a1c7cda694a733ccbc4cd77e84c780a01099...  20:58   
..                                                ...    ...   
45  1cd913b8855ec7cfdc2a065431ec5162826398e3e05574...  20:59   
46  ab611a00fa8f423e874b609ee1da63463e9496b9cc30f1...  20:59   
47  05ad819b0af2d0fe93a81f3077b15254dfe2c44f6221c9...  20:59   
48  f9a0127941eaeb00af30e0ea049c0aca6f454dd6ee4381...  20:59   
49  4eb37337ec11c7c3519dfc3536ef43d09e6c1f2d797bbd...  20:59   

           BTC-value       USD-value  
0     0.00067150 BTC          $25.75  
1     0.03594438 BTC       $1,378.60  
2     2.02645842 BTC      $77,722.26  
3   262.08955852 BTC  $10,052,114.78  
4     0.34455132 BTC      $13,214.83

In [6]:
#Cleanen van database
liveCall.appendedDF['BTC-value'] = liveCall.appendedDF['BTC-value'].map(lambda x: x.rstrip(' BTC'))
liveCall.appendedDF['BTC-value'] = liveCall.appendedDF['BTC-value'].str.replace(r'\,', '')
#liveCall.appendedDF['BTC-value'] = liveCall.appendedDF['BTC-value'].str.replace(r'\.', ',')

liveCall.appendedDF['USD-value'] = liveCall.appendedDF['USD-value'].map(lambda x: x.lstrip('$'))
liveCall.appendedDF['USD-value'] = liveCall.appendedDF['USD-value'].str.replace(r'\,', '')
#liveCall.appendedDF['USD-value'] = liveCall.appendedDF['USD-value'].str.replace(r'\.', ',')

liveCall.appendedDF['USD-value'] = liveCall.appendedDF['USD-value'].astype('float')
liveCall.appendedDF['BTC-value'] = liveCall.appendedDF['BTC-value'].astype('float')

#
# liveCall.appendedDF.loc[liveCall.appendedDF['Time'] == '20:34', 'USD-value']

dtype('float64')

In [27]:
#Haal eruit welke het grootste is
timestamps = liveCall.appendedDF['Time'].unique()

f = open("logfile.txt", "w")
for timestamp in timestamps:
    BTC = liveCall.appendedDF.loc[liveCall.appendedDF['Time'] == timestamp, 'BTC-value'].max()
    series = liveCall.appendedDF.loc[liveCall.appendedDF['BTC-value'] == BTC]
    
    f.write(f"============ Hier zijn alle gegevens voor de max-BTC-value van {timestamp}: ")
    f.write("\n")
    for names in series.columns:
        if names != "Time":
            f.write(f"De {names}: {series.iloc[0][names]}")
            f.write("\n")
    
    f.write("\n")

f.close()